In [1]:
import jax
import jax.numpy as jnp
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm
import cv2
from glob import glob

from vit_jax import models

2024-11-19 16:24:06.922408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732013646.935413   36404 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732013646.939248   36404 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
[name for name in models.model_configs.MODEL_CONFIGS if name.startswith('LiT')]

['LiT-B16B', 'LiT-B16B_2', 'LiT-L16L', 'LiT-L16S', 'LiT-L16Ti']

In [3]:
model_name = 'LiT-B16B'

lit_model = models.get_model(model_name)
# Loading the variables from cloud can take a while the first time...
lit_variables = lit_model.load_variables()
# Creating tokens from freeform text (see next section).
tokenizer = lit_model.get_tokenizer()
# Resizing images & converting value range to -1..1 (see next section).
image_preprocessing = lit_model.get_image_preprocessing()
# Preprocessing op for use in tfds pipeline (see last section).
pp = lit_model.get_pp()


⚠️ Reusing local copy: LiT-B16B.npz


2024-11-19 16:24:24.534366: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-11-19 16:24:24.584877: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


In [4]:
print(lit_model)
type(lit_model)

LitModel(
    # attributes
    image = classifier: token
    hidden_size: 768
    model_name: ViT-B_16
    patches:
      size: !!python/tuple
      - 16
      - 16
    representation_size: null
    transformer:
      attention_dropout_rate: 0.0
      dropout_rate: 0.0
      mlp_dim: 3072
      num_heads: 12
      num_layers: 12
    
    text_model = 'bert'
    text = config: base
    
    pp = max_len: 16
    size: 224
    tokenizer_name: bert
    
    out_dim = (768, 768)
    model_name = 'LiT-B16B'
)


vit_jax.models_lit.LitModel

In [5]:
lit_model.out_dim

(768, 768)

In [6]:
from vit_jax.models_vit import VisionTransformer
image_model = VisionTransformer(
    **{
        **lit_model.image,
        'num_classes': lit_model.out_dim[0],
    }, name='img'
)
image_model

VisionTransformer(
    # attributes
    num_classes = 768
    patches = size: !!python/tuple
    - 16
    - 16
    
    transformer = attention_dropout_rate: 0.0
    dropout_rate: 0.0
    mlp_dim: 3072
    num_heads: 12
    num_layers: 12
    
    hidden_size = 768
    resnet = None
    representation_size = None
    classifier = 'token'
    head_bias_init = 0.0
    encoder = Encoder
    model_name = 'ViT-B_16'
)

In [8]:
from orbax.export import ExportManager
from orbax.export import JaxModule
from orbax.export import ServingConfig
import tensorflow as tf
import jax.numpy as jnp

jax_module = JaxModule({}, image_model, input_polymorphic_shape='b, ...')
jax_module

In [12]:
class MyJaxModule(JaxModule):
    def __hash__(self):
        return id(self)
my_jax_module = JaxModule({}, image_model, input_polymorphic_shape='b, ...')
my_jax_module

In [10]:
from orbax.export import constants
tf.saved_model.save(
    jax_module,
    './vit_tf_save',
    signatures=jax_module.methods[constants.DEFAULT_METHOD_KEY].get_concrete_function(
        tf.TensorSpec(shape=(None,), dtype=tf.float32, name="input")
    ),
    options=tf.saved_model.SaveOptions(experimental_custom_gradients=True),
)

TypeError: Failed to hash Flax Module.  The module probably contains unhashable attributes.  Module=VisionTransformer(
    # attributes
    num_classes = 768
    patches = size: !!python/tuple
    - 16
    - 16
    
    transformer = attention_dropout_rate: 0.0
    dropout_rate: 0.0
    mlp_dim: 3072
    num_heads: 12
    num_layers: 12
    
    hidden_size = 768
    resnet = None
    representation_size = None
    classifier = 'token'
    head_bias_init = 0.0
    encoder = Encoder
    model_name = 'ViT-B_16'
)

In [13]:
from orbax.export import constants
tf.saved_model.save(
    jax_module,
    './vit_tf_save',
    signatures=my_jax_module.methods[constants.DEFAULT_METHOD_KEY].get_concrete_function(
        tf.TensorSpec(shape=(None,), dtype=tf.float32, name="input")
    ),
    options=tf.saved_model.SaveOptions(experimental_custom_gradients=True),
)

TypeError: Failed to hash Flax Module.  The module probably contains unhashable attributes.  Module=VisionTransformer(
    # attributes
    num_classes = 768
    patches = size: !!python/tuple
    - 16
    - 16
    
    transformer = attention_dropout_rate: 0.0
    dropout_rate: 0.0
    mlp_dim: 3072
    num_heads: 12
    num_layers: 12
    
    hidden_size = 768
    resnet = None
    representation_size = None
    classifier = 'token'
    head_bias_init = 0.0
    encoder = Encoder
    model_name = 'ViT-B_16'
)